In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
train_test_split = 0.05
batch_size = 10
learning_rate = 0.001
keep_prob = 0.7

epochs = 1001
validate_every_n_epochs = 100

pretrained_model = 'models/pretrained'
model_path = 'models/mnist'
logs_path = 'logs'

In [ ]:
train_df = pd.read_csv('data/digit-recognizer/train.csv')
test_df = pd.read_csv('data/digit-recognizer/test.csv')

In [ ]:
train_labels, train_features = train_df.values[:, 0], train_df.values[:, 1:]
test_features = test_df.values.astype(np.float32)

In [ ]:
train_indices = list(range(train_features.shape[0]))
np.random.shuffle(train_indices)
val_size = int(len(train_indices) * train_test_split)
val_indices, train_indices = train_indices[:val_size], train_indices[val_size:]

In [ ]:
val_labels, train_labels = train_labels[val_indices,], train_labels[train_indices]
val_features, train_features = train_features[val_indices,:], train_features[train_indices,:]

In [ ]:
def one_hot_encode(x, set):
    if x not in set:
        raise ValueError("Element {} not in the available values".format(x))
    return list(map(lambda s: int(x == s), set))

classes = sorted(set(train_labels))

train_labels = np.array([one_hot_encode(label, classes) for label in train_labels]).astype(np.float32)
train_features = train_features.astype(np.float32)

val_labels = np.array([one_hot_encode(label, classes) for label in val_labels]).astype(np.float32)
val_features = val_features.astype(np.float32)


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).batch(batch_size)

In [ ]:
train_iterator = train_dataset.make_one_shot_iterator()
val_iterator = val_dataset.make_one_shot_iterator()

In [ ]:
num_test_batches = test_features.shape[0] // batch_size + 1

if test_features.shape[0] % batch_size == 0:
    num_test_batches -= 1

def get_next_test_batch(batch_num, features):
    start_index = batch_num * batch_size
    assert start_index <= features.shape[0]
    
    end_index = min((batch_num + 1) * batch_size, features.shape[0])
    return test_features[start_index:end_index, :]

In [ ]:
def init_weights(name, shape):
    """
    Initializes weights
    Param:
        name: string. name of the global variable
        shape: list of int. shape of the global_variable
    Returns:
        tf.Variable
    """
    return tf.get_variable(name=name, shape=shape, dtype=tf.float32,
                          initializer=tf.initializers.glorot_uniform)

def conv2d(x, W):
    """
    Applies filter W to input x with padding 'SAME' and stride of 1
    """
    return tf.nn.conv2d(x, W, padding='SAME', strides=[1, 1, 1, 1])

def max_pool_2by2(x):
    """
    Applies max_pool with strides [2, 2] and kernel size [2, 2]
    """
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                         padding='SAME')

In [ ]:
"""
Model Definition
"""

num_channels_1 = 32
kernel_size_1 = 5

num_channels_2 = 64
kernel_size_2 = 3

n_dense = 128
n_class = len(classes)

x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, n_class])

x_reshaped = tf.reshape(x, [-1, 28, 28, 1])

filter_1 = init_weights(
    'filter_1', [kernel_size_1, kernel_size_1, 1, num_channels_1])
bias_1 = init_weights('bias_1', [num_channels_1])
conv_1 = tf.nn.relu(conv2d(x_reshaped, filter_1) + bias_1)

filter_2 = init_weights(
    'filter_2', [kernel_size_2, kernel_size_2, num_channels_1, num_channels_2])
bias_2 = init_weights('bias_2', [num_channels_2])
conv_2 = tf.nn.relu(conv2d(conv_1, filter_2) + bias_2)

conv_2_shape = conv_2.get_shape().as_list()
flattened_size = np.prod(conv_2_shape[1:])
x_flattened = tf.reshape(conv_2, [-1, flattened_size])

W_1 = init_weights('W_1', [flattened_size, n_dense])
b_1 = init_weights('b_1', [n_dense])
h_1 = tf.nn.relu(tf.matmul(x_flattened, W_1) + b_1)
h_1 = tf.nn.dropout(h_1, keep_prob=keep_prob)

W_2 = init_weights('W_2', [n_dense, n_class])
b_2 = init_weights('b_2', [n_class])
y_pred = tf.matmul(h_1, W_2) + b_2 

In [ ]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_pred))
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [ ]:
def calculate_accuracy(y_true, y_hat):
    scores = tf.equal(tf.argmax(y_true, 1), tf.argmax(y_hat, 1))
    return tf.reduce_mean(tf.cast(scores, tf.float32))
accuracy = calculate_accuracy(y, y_pred)

In [ ]:
writer = tf.summary.FileWriter(logs_path, tf.get_default_graph())
tf.summary.scalar('training_loss', loss)
tf.summary.scalar('validation_accuracy', accuracy)
summaries = tf.summary.merge_all()

In [ ]:
labels = []

tf.add_to_collection(
    'ops_to_restore', tf.get_default_graph().get_operations())
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #if pretrained_model:
    #    restorer = tf.train.import_meta_graph(pretrained_model + '.meta')
    #    restorer.restore(sess, pretrained_model)

    
    for epoch in range(epochs):
        x_batch, y_batch = sess.run(train_iterator.get_next())
        _, loss_epoch, summary_proto = sess.run(
            [train, loss, summaries], feed_dict={x: x_batch, y: y_batch})
        writer.add_summary(summary_proto, global_step=epoch)
        print('Epoch {}: Training Loss -'.format(epoch), end=' ')
        print(loss_epoch)
        
        if epoch % validate_every_n_epochs == 0:
            x_val_batch, y_val_batch = sess.run(val_iterator.get_next())
            accuracy_epoch, summary_proto = sess.run(
                [accuracy, summaries], feed_dict={x: x_val_batch, y: y_val_batch})
            writer.add_summary(summary_proto, global_step=epoch)
            saver.save(sess, model_path, global_step=epoch)
            print('Epoch {}: Validation Accuracy -'.format(epoch), end=' ')
            print(accuracy_epoch)
            
    for j in range(num_test_batches):
        x_test = get_next_test_batch(j, test_features).astype(np.float32)
        dummy_y = np.zeros([batch_size, n_class]).astype(np.float32)
        test_predicted = sess.run(
            y_pred, feed_dict={x: tf.constant(x_test), y: tf.constant(dummy_y)})
        
        labels.extend(test_predicted.tolist())

In [ ]:
#tf.reset_default_graph()
labels = []
#graph = tf.get_default_graph()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph(pretrained_model + '.meta')
    saver.restore(sess, pretrained_model)

    for j in range(num_test_batches):
        x_test = get_next_test_batch(j, test_features).astype(np.float32)
        dummy_y = np.zeros([batch_size, n_class]).astype(np.float32)
        test_predicted = sess.run(y_pred, feed_dict={x: x_test, y: dummy_y})

        labels.extend(np.argmax(test_predicted, 1).tolist())

In [ ]:
from collections import Counter
len(labels)